### Import all libraries

In [31]:
import pandas as pd
import yfinance as yf
import numpy as np

## STEP 1 = Discount Rate

#### Calculate Cost of Equity using Capital Asset Pricing Model

In [2]:
def get_risk_free_rate():
    url = "https://finance.yahoo.com/bonds"
    us_bonds = pd.read_html(url)    
    us_bonds_df = us_bonds[0]
    index = ["13_week_tbill", "treasury_yield_5years", "treasury_yield_10years", "treasury_yield_30years"]
    us_bonds_df = us_bonds_df.set_index("Name")
    us_10_year_bond_yield = (us_bonds_df.loc["Treasury Yield 10 Years", "Last Price"])/100
    return us_10_year_bond_yield

In [3]:
get_risk_free_rate()

0.03449

In [4]:
# Define a function to get stock beta using Yahoo Finance library

def stock_beta(ticker_value):
    ticker_data = yf.Ticker(ticker_value)
    ticker_beta = ticker_data.info["beta"]
    return ticker_beta

In [5]:
stock_beta("MSFT")

0.926403

In [6]:
# Define function to calculate the Cost of Equity (we use Capital Asset Pricing Model)

def cost_of_equity(beta):
    risk_free_rate = get_risk_free_rate() # Higher T-Bill yield will result in higher stock return...  The current 10-Y US T-Bill rate is 3.87%, which is quite high because the fed keeps increasing returns to combat inflation. But to be more realistic I took a rate of 2.5%
    expected_market_return = 0.08 # Higher expected market rate is giving higher expected stock return... I am taking a conservative value of 8%
    expected_stock_return = risk_free_rate + (beta*(expected_market_return - risk_free_rate))
    return expected_stock_return

In [7]:
cost_of_equity(stock_beta("MSFT"))

0.07665060053

#### Get the weights assigned to each source of capital

In [8]:
# Define function to get Total Debt value from Balance Sheet

def get_total_debt(ticker_value):
    # Set the stock ticker
    stock_ticker = yf.Ticker(ticker_value)
    
    # Get the Balance Sheet data of the stock ticker in a Pandas DataFrame
    stock_balance_sheet = stock_ticker.balance_sheet
    
    # Get the value of the latest 10K filling date
    cols = stock_balance_sheet.columns
    latest_filling_date = cols[0]
    latest_filling_date = latest_filling_date.strftime("%Y-%m-%d")
    
    # Get value for Total Debt
    total_debt = stock_balance_sheet.loc["Total Debt", latest_filling_date]
    return round(total_debt)

In [40]:
get_total_debt("MSFT")

61270000000

In [9]:
def get_market_cap(ticker_value):
    
    # Set the stock ticker
    stock_ticker = yf.Ticker(ticker_value)
    
    # Get the Balance Sheet data of the stock ticker in a Pandas DataFrame
    market_cap = stock_ticker.info["marketCap"]
        
    return market_cap

In [10]:
get_market_cap("MSFT")

1777965596672

In [11]:
# Define function to calculate weight of equity
def weight_of_equity(ticker_value):
    equity_weight = get_market_cap(ticker_value) / ( get_total_debt(ticker_value) +  get_market_cap(ticker_value))
    return equity_weight


# Define function to calculate weight of debt
def weight_of_debt(ticker_value):
    debt_weight = get_total_debt(ticker_value) / ( get_total_debt(ticker_value) +  get_market_cap(ticker_value))
    return debt_weight

In [12]:
weight_of_equity("MSFT")

0.9666872476202262

In [14]:
weight_of_debt("MSFT")

0.03331275237977388

#### Calculate Cost of Debt

In [15]:
# Define function to calculate Cost of debt

def cost_of_debt(ticker_value):
    # Set the stock ticker
    stock_ticker = yf.Ticker(ticker_value)
    
    # Get the Income Statement data of the stock ticker in a Pandas DataFrame
    stock_income_statement = stock_ticker.income_stmt
    
    # Get the value of the latest 10K filling date
    cols = stock_income_statement.columns
    latest_filling_date = cols[0]
    latest_filling_date = latest_filling_date.strftime("%Y-%m-%d")
    
    # Get value for Interest expense from Income Statement
    interest_expense = stock_income_statement.loc["Interest Expense Non Operating",latest_filling_date]
    #print(interest_expense)

    # Get value of total debt from get_total_debt() function
    total_debt = get_total_debt(ticker_value)

    # Calculate Cost of Debt = (Interest Expense) / (Total Debt)
    debt_cost = interest_expense / total_debt

    return debt_cost

In [17]:
cost_of_debt("MSFT")

0.0336706381589685

#### Calculate Effective Tax Rate

In [18]:
# Define function to calculate Effective Tax Rate

# Effective Tax Rate = (Income Tax Expense) / (Pre-tax Income)
 

def effective_tax_rate(ticker_value):
    # Set the stock ticker
    stock_ticker = yf.Ticker(ticker_value)
    
    # Get the Income Statement data of the stock ticker in a Pandas DataFrame
    stock_income_statement = stock_ticker.income_stmt
    
    # Get the value of the latest 10K filling date
    cols = stock_income_statement.columns
    latest_filling_date = cols[0]
    latest_filling_date = latest_filling_date.strftime("%Y-%m-%d")
    
    # Get value for Income Tax Expense from Income Statement
    income_tax_expense = stock_income_statement.loc["Tax Provision", latest_filling_date]
    #print("The income tax expense is: " + str(income_tax_expense))
    #print(type(income_tax_expense))
    #print("_________________________________________")

    # Get value for Pretax Income from Income Statement
    pretax_income = stock_income_statement.loc["Pretax Income", latest_filling_date]
    #print("The pretax income is: " + str(pretax_income))
    #print(type(pretax_income))

    # Calculate Effective Tax Rate = (Income Tax Expense) / (Pretax Income)
    effective_tax_rate = income_tax_expense / pretax_income


    return effective_tax_rate

In [41]:
effective_tax_rate("MSFT")

0.13113383343685794

#### Calculate WACC

In [20]:
# Define function to calculate WACC
def wacc(ticker_value):
    weight_equity = weight_of_equity(ticker_value)
    cost_equity = cost_of_equity(stock_beta(ticker_value))
    weight_debt = weight_of_debt(ticker_value)
    cost_debt = cost_of_debt(ticker_value)
    eff_tax_rate_minus_1 = (1 - effective_tax_rate(ticker_value))
    discount_rate = (weight_equity * cost_equity) + (weight_debt * cost_debt * eff_tax_rate_minus_1)
    return discount_rate

In [22]:
wacc("MSFT")

0.07507173189668961

### STEP 2 = Project Free Cash Flows and Terminal Value

#### Calculate the expected growth rate of the Free Cash Flows

In [23]:
# Define function to calculate Average Free Cash Flow Growth Rate

def fcf_growth_rate(ticker_value):
    
    # Get the ticker value from the user
    ticker = yf.Ticker(ticker_value)
    
    # Get the Statement of Cash Flows from Yahoo Finance
    ticker_cash_flow_statement = ticker.cashflow
    
    # Get the historical Free Cash Flow values from the Cash Flow Statement
    ticker_historical_fcf = ticker_cash_flow_statement.loc["Free Cash Flow", :]
    
    # Get the Free Cash Flow values into a Pandas DataFrame
    ticker_historical_fcf_df = pd.DataFrame(ticker_historical_fcf)
    
    # Sort the indices of the Free Cash Flow values DataFrame to see the oldest FCF value first
    ticker_historical_fcf_df = ticker_historical_fcf_df.sort_index()
    #display(ticker_historical_fcf_df)
    
    # Create a new column that contains the FCF Growth Rate (using pct_change() function)
    ticker_historical_fcf_df["FCF Growth Rate"] = ticker_historical_fcf_df.pct_change(periods=1)
    #display(ticker_historical_fcf_df)
    
    # Get the minimum value for the FCF Growth Rate, which will be used to project the future Free Cash Flows. We use the minimum value for the FCF Growth Rate to get the most conservative estimate.
    fcf_growth_rate = ticker_historical_fcf_df["FCF Growth Rate"].min()
    #print(fcf_growth_rate)
    
    # Return the Free Cash Flow Growth Rate
    return fcf_growth_rate

In [24]:
fcf_growth_rate("MSFT")

0.1609287572614848

#### Project the Free Cash Flows and the Terminal Value

In [25]:
def projected_fcf(ticker_value):
    
    # Get the growth rate for the specified company ticker using fcf_growth_rate() function
    growth_rate = fcf_growth_rate(ticker_value)
    
    # Hard-code the perpetual growth rate for the company (equivalent to US Economy growth rate)
    perpetual_growth_rate = 0.02
    
    # Get the discount rate (WACC) for the specified company ticker using wacc() function
    discount_rate_wacc = wacc(ticker_value)
    
    # Get the ticker value as function's parameter
    ticker = yf.Ticker(ticker_value)

    # Get the Statement of Cash Flows from Yahoo Finance
    ticker_cash_flow_statement = ticker.cashflow
    #print(msft_cash_flow_stmt.loc["Free Cash Flow", :])

    
    # Get the historical Free Cash Flow values from the Cash Flow Statement
    ticker_historical_fcf_df = pd.DataFrame(ticker_cash_flow_statement.loc["Free Cash Flow", :])
    #display(ticker_historical_fcf_df)
    
    # Sort the indices of the historical FCF DataFrame to see the oldest FCF value first
    ticker_historical_fcf_df = ticker_historical_fcf_df.sort_index()
    #display(ticker_historical_fcf_df)
    
    # Reset the indices of historical FCF DataFrame
    ticker_historical_fcf_df = ticker_historical_fcf_df.reset_index()
    
    # Create a new DataFrame column called "Year"
    ticker_historical_fcf_df['Year'] = pd.DatetimeIndex(ticker_historical_fcf_df['index']).year
    
    # Drop the index column from historical FCF DataFrame
    ticker_historical_fcf_df = ticker_historical_fcf_df.drop(columns="index")
    #display(ticker_historical_fcf_df)
    
    # Get latest FCF value from the historical FCF DataFrame. This is our starting point for projecting FCF.
    latest_fcf_df = pd.DataFrame(ticker_historical_fcf_df.iloc[(len(ticker_historical_fcf_df) - 1), :]).T.reset_index(drop=True)
    #display(latest_fcf_df)
        
    most_recent_year = latest_fcf_df["Year"][0]
    #display(most_recent_year)
        
    # Project first FCF, one year into the future
    projected_fcf_1 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Free Cash Flow"] * (1+growth_rate)
    projected_year_1 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Year"] + 1
    latest_fcf_df.loc[len(latest_fcf_df.index)] = [projected_fcf_1, projected_year_1]
    #display(latest_fcf_df)

    # Project second FCF, two years into the future
    projected_fcf_2 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Free Cash Flow"] * (1+growth_rate)
    projected_year_2 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Year"] + 1
    latest_fcf_df.loc[len(latest_fcf_df.index)] = [projected_fcf_2, projected_year_2]
    #display(latest_fcf_df)
    
    # Project third FCF, three years into the future
    projected_fcf_3 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Free Cash Flow"] * (1+growth_rate)
    projected_year_3 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Year"] + 1
    latest_fcf_df.loc[len(latest_fcf_df.index)] = [projected_fcf_3, projected_year_3]
    #display(latest_fcf_df)
    
    # Project fourth FCF, four years into the future
    projected_fcf_4 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Free Cash Flow"] * (1+growth_rate)
    projected_year_4 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Year"] + 1
    latest_fcf_df.loc[len(latest_fcf_df.index)] = [projected_fcf_4, projected_year_4]

                        
                        
    #display(latest_fcf_df)
    
    # Project fifth FCF, five years into the future
    projected_fcf_5 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Free Cash Flow"] * (1+growth_rate)
    projected_year_5 = latest_fcf_df.loc[(len(latest_fcf_df.index)-1), "Year"] + 1
    latest_fcf_df.loc[len(latest_fcf_df.index)] = [projected_fcf_5, projected_year_5]
    #print(latest_fcf_df)
    
    
    # Calculate the terminal value
    
    # Get the last projected FCF, five years into the future
    last_projected_fcf = latest_fcf_df.loc[(len(latest_fcf_df)-1), "Free Cash Flow"]
    #print(last_projected_fcf)
    
    # Get the last projected Year, five years into the future
    last_projected_year = latest_fcf_df.loc[(len(latest_fcf_df)-1), "Year"]
    
    # Calculate the terminal value. Formula: {(Last projected FCF * (1 + Perpetual growth rate)} / {(WACC discount rate) - (Perpetual growth rate)}
    projected_terminal_value = (last_projected_fcf * (1 + perpetual_growth_rate)) / (discount_rate_wacc - perpetual_growth_rate)
    #print(projected_terminal_value)
    
    # Add the terminal value to the latest_fcf_df DataFrame
    latest_fcf_df.loc[len(latest_fcf_df.index)] = [projected_terminal_value, last_projected_year]
    #print(latest_fcf_df)
    
    # Grab the calculated terminal value from the last row of the latest_fcf_df DataFrame
    terminal_value = latest_fcf_df.loc[(len(latest_fcf_df.index) - 1), "Free Cash Flow"]
    
    # Drop the last row of the latest_fcf_df DataFrame
    latest_fcf_df = latest_fcf_df.drop(latest_fcf_df.tail(1).index) # drop last 1 row
    #print(latest_fcf_df)
    
    # Add the calculated terminal value to the final projected FCF, five years into the future
    latest_fcf_df.loc[len(latest_fcf_df) - 1] = [(projected_fcf_5 + terminal_value), projected_year_5]
    
    # Drop the first row from the latest_fcf_df DataFrame
    latest_fcf_df = latest_fcf_df.drop(latest_fcf_df.head(1).index) # drop first 1 row

    # Drop the "Year" column from the latest_fcf_df DataFrame
    latest_fcf_df = latest_fcf_df.drop(columns="Year").reset_index(drop=False)
    
    return latest_fcf_df


In [26]:
projected_fcf("MSFT")

,index,Free Cash Flow
0,1,75633347606.828476
1,2,87804928244.721283
2,3,101935266228.578125
3,4,118339581943.861801
4,5,2681910669984.48291


## Steps 3 & 4 - Calculate present values of FCF & Terminal Value, and add up all present values

In [27]:
# Define a function to calculate the sum of Present Values of all projected FCF

def present_values(ticker_value):
    
    # Get the discount rate, wacc() function
    discount_rate = wacc(ticker_value)
    
    # Get the Data Frame that contains the projected future cash flows, projected_fcf() function
    projected_fcf_df = projected_fcf(ticker_value)
    
    # Create an empty column called "Present Value" in the projected FCF DataFrame
    projected_fcf_df["Present Value"] = np.nan

    # Create a FOR loop that will loop through each row of projected FCF Data Frame, calculating the present value of each projected cash flow
    for each_index, each_row in projected_fcf_df.iterrows():
        projected_fcf_df.loc[each_index, "Present Value"] = (projected_fcf_df.loc[each_index, "Free Cash Flow"]) / ( (1 + discount_rate)**(projected_fcf_df.loc[each_index, "index"]) )
    
    
    #print(projected_fcf_df)
    # Add up the Present Values of all projected cash flows. This is the Enterprise Value
    enterprise_value = projected_fcf_df["Present Value"].sum()
    
    # Return the Enterprise Value
    return enterprise_value


In [34]:
present_values("MSFT")

2184433661147.0425

## Steps 5 & 6 - Calculate the Equity Value, and finally the Intrinsic Value per share

In [35]:
def intrinsic_value(ticker_value):
    
    # Get the Enterprise Value, pv() function
    enterprise_value = present_values(ticker_value)
    #print(enterprise_value)
    
    # Set the stock ticker
    stock_ticker = yf.Ticker(ticker_value)
    
    # Get the Balance Sheet data of the stock ticker in a Pandas DataFrame
    stock_balance_sheet = stock_ticker.balance_sheet
    
    # Get the value of the latest 10K filling date
    cols = stock_balance_sheet.columns
    latest_filling_date = cols[0]
    latest_filling_date = latest_filling_date.strftime("%Y-%m-%d")
    
    # Get value for "Cash, Cash Equivalents, And Short Term Investments"
    cash = stock_balance_sheet.loc["Cash Cash Equivalents And Short Term Investments", latest_filling_date]
    #print(cash)
    
    # Get value for total debt, get_total_debt() function
    total_debt = get_total_debt(ticker_value)
    #print(total_debt)
    
    # Calculate Equity Value (= Enterprise Value + Cash - Total Debt)
    equity_value = enterprise_value + cash - total_debt
    #print(equity_value)
    
    # Get no. of shares outstanding
    shares_outstanding = stock_ticker.info["sharesOutstanding"]
    #print(shares_outstanding)
    
    # Calculate intrinsic value per share (= equity_value / shares_outstanding)
    intrinsic_value_per_share = equity_value / shares_outstanding
    #print(intrinsic_value_per_share)
    
    return intrinsic_value_per_share

In [39]:
intrinsic_value("MSFT")

298.86935196061637